# Comprehensive Multilayer Viral Infection Model
 

 This notebook models a complete viral infection system with:
 
 **Biological Layers:**
- **Host cell signaling** (protein interactions)
 - **Viral replication machinery** (viral proteins + host factors)
 - **Immune response** (cytokines, receptors, immune cells)
 - **Metabolic hijacking** (virus reprograms host metabolism)
 - **Transcriptional regulation** (TFs, viral modulators)

In [2]:
# ## 1. Setup & Initialization

import numpy as np
import polars as pl
from collections import defaultdict
from scipy.sparse.linalg import eigsh
import json
import time

import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import Graph

In [7]:
# ## 2. Initialize Graph with Pre-allocation & History

# pre-allocate for performance (estimated size)
G = Graph(directed=None, n=200, e=500)  # mixed directionality

# enable full history tracking
G.enable_history(True)
G.mark("initialization_start")

# set composite vertex key for entity lookup
G.set_vertex_key("entity_type", "name")

print(f"Graph initialized with capacity: {G._matrix.shape}")
print(f"History enabled: {G._history_enabled}")

Graph initialized with capacity: (200, 500)
History enabled: True


In [8]:
# ## 3. Define Multilayer Structure (5 Aspects)

G.set_aspects(
    aspects=["biological_context"],
    elem_layers={
        "biological_context": [
            "host_signaling",
            "viral_replication", 
            "immune_response",
            "metabolic",
            "transcriptional"
        ]
    }
)

# set aspect-level metadata
G.set_aspect_attrs("biological_context", 
    description="Biological layer of interaction",
    organism="human + SARS-CoV-2-like virus"
)

# set layer-level metadata
layer_metadata = {
    ("host_signaling",): {"cell_type": "epithelial", "compartment": "cytoplasm"},
    ("viral_replication",): {"compartment": "cytoplasm + ER", "timescale": "hours"},
    ("immune_response",): {"cell_types": ["macrophage", "T-cell", "dendritic"], "systemic": True},
    ("metabolic",): {"pathways": ["glycolysis", "TCA", "lipid_synthesis"], "compartment": "mitochondria + cytoplasm"},
    ("transcriptional",): {"compartment": "nucleus", "regulation": "bidirectional"},
}

for layer_tuple, attrs in layer_metadata.items():
    G.set_layer_attrs(layer_tuple, **attrs)

print("Layers configured:")
print(G.layers_view())

Layers configured:
shape: (5, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬──────────┬───────────┬───────────┐
│ layer_tup ┆ layer_id  ┆ biologica ┆ cell_type ┆ … ┆ cell_type ┆ systemic ┆ pathways  ┆ regulatio │
│ le        ┆ ---       ┆ l_context ┆ ---       ┆   ┆ s         ┆ ---      ┆ ---       ┆ n         │
│ ---       ┆ str       ┆ ---       ┆ str       ┆   ┆ ---       ┆ bool     ┆ list[str] ┆ ---       │
│ list[str] ┆           ┆ str       ┆           ┆   ┆ list[str] ┆          ┆           ┆ str       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪══════════╪═══════════╪═══════════╡
│ ["host_si ┆ host_sign ┆ host_sign ┆ epithelia ┆ … ┆ null      ┆ null     ┆ null      ┆ null      │
│ gnaling"] ┆ aling     ┆ aling     ┆ l         ┆   ┆           ┆          ┆           ┆           │
│ ["viral_r ┆ viral_rep ┆ viral_rep ┆ null      ┆ … ┆ null      ┆ null     ┆ null      ┆ null      │
│ eplicatio ┆ lication  ┆ lication  ┆           ┆   ┆    

In [9]:
# ## 4. Create Biological Entities
# 
# ### 4.1 Host Proteins

host_proteins = [
    # receptor and entry
    {"name": "ACE2", "function": "viral_receptor", "location": "membrane"},
    {"name": "TMPRSS2", "function": "protease", "location": "membrane"},
    {"name": "FURIN", "function": "protease", "location": "golgi"},
    
    # signaling cascade
    {"name": "STAT1", "function": "transcription_factor", "location": "cytoplasm"},
    {"name": "STAT2", "function": "transcription_factor", "location": "cytoplasm"},
    {"name": "IRF3", "function": "transcription_factor", "location": "cytoplasm"},
    {"name": "IRF7", "function": "transcription_factor", "location": "cytoplasm"},
    {"name": "NFkB", "function": "transcription_factor", "location": "cytoplasm"},
    {"name": "MAVS", "function": "adaptor", "location": "mitochondria"},
    {"name": "RIG_I", "function": "sensor", "location": "cytoplasm"},
    {"name": "MDA5", "function": "sensor", "location": "cytoplasm"},
    {"name": "TBK1", "function": "kinase", "location": "cytoplasm"},
    {"name": "IKKe", "function": "kinase", "location": "cytoplasm"},
    
    # metabolic enzymes
    {"name": "HK2", "function": "glycolysis", "location": "cytoplasm"},
    {"name": "PFKFB3", "function": "glycolysis", "location": "cytoplasm"},
    {"name": "PKM2", "function": "glycolysis", "location": "cytoplasm"},
    {"name": "LDHA", "function": "glycolysis", "location": "cytoplasm"},
    {"name": "ACC1", "function": "lipid_synthesis", "location": "cytoplasm"},
    {"name": "FASN", "function": "lipid_synthesis", "location": "cytoplasm"},
    {"name": "SCD1", "function": "lipid_synthesis", "location": "ER"},
    
    # translation machinery
    {"name": "eIF4E", "function": "translation", "location": "cytoplasm"},
    {"name": "eIF4G", "function": "translation", "location": "cytoplasm"},
    {"name": "PABP", "function": "translation", "location": "cytoplasm"},
    {"name": "RPL4", "function": "ribosome", "location": "cytoplasm"},
]

for protein in host_proteins:
    G.add_vertex(
        f"host_{protein['name']}", 
        entity_type="host_protein",
        name=protein["name"],
        function=protein["function"],
        location=protein["location"],
        is_viral=False
    )

print(f"Added {len(host_proteins)} host proteins")

Added 24 host proteins


In [10]:
# ### 4.2 Viral Proteins

viral_proteins = [
    # structural
    {"name": "Spike", "function": "entry", "essential": True},
    {"name": "Membrane", "function": "structure", "essential": True},
    {"name": "Envelope", "function": "structure", "essential": True},
    {"name": "Nucleocapsid", "function": "RNA_binding", "essential": True},
    
    # non-structural (replication)
    {"name": "NSP1", "function": "host_shutoff", "essential": True},
    {"name": "NSP3", "function": "protease", "essential": True},
    {"name": "NSP5", "function": "main_protease", "essential": True},
    {"name": "NSP12", "function": "RdRp", "essential": True},  # RNA-dependent RNA polymerase
    {"name": "NSP13", "function": "helicase", "essential": True},
    {"name": "NSP14", "function": "exonuclease", "essential": True},
    {"name": "NSP15", "function": "endoribonuclease", "essential": False},
    {"name": "NSP16", "function": "methyltransferase", "essential": True},
    
    # accessory (immune evasion)
    {"name": "ORF3a", "function": "ion_channel", "essential": False},
    {"name": "ORF6", "function": "immune_evasion", "essential": False},
    {"name": "ORF7a", "function": "immune_evasion", "essential": False},
    {"name": "ORF8", "function": "immune_evasion", "essential": False},
]

for protein in viral_proteins:
    G.add_vertex(
        f"viral_{protein['name']}",
        entity_type="viral_protein",
        name=protein["name"],
        function=protein["function"],
        essential=protein["essential"],
        is_viral=True
    )

print(f"Added {len(viral_proteins)} viral proteins")

Added 16 viral proteins


In [11]:
# ### 4.3 Immune Molecules

immune_molecules = [
    # cytokines
    {"name": "IFN_alpha", "type": "cytokine", "class": "type_I_IFN"},
    {"name": "IFN_beta", "type": "cytokine", "class": "type_I_IFN"},
    {"name": "IFN_gamma", "type": "cytokine", "class": "type_II_IFN"},
    {"name": "IL6", "type": "cytokine", "class": "proinflammatory"},
    {"name": "IL1beta", "type": "cytokine", "class": "proinflammatory"},
    {"name": "TNF_alpha", "type": "cytokine", "class": "proinflammatory"},
    {"name": "IL10", "type": "cytokine", "class": "antiinflammatory"},
    {"name": "CXCL10", "type": "chemokine", "class": "chemoattractant"},
    
    # receptors
    {"name": "IFNAR1", "type": "receptor", "class": "type_I_IFN_receptor"},
    {"name": "IFNAR2", "type": "receptor", "class": "type_I_IFN_receptor"},
    {"name": "TLR3", "type": "receptor", "class": "pattern_recognition"},
    {"name": "TLR7", "type": "receptor", "class": "pattern_recognition"},
    
    # immune cell markers (abstracted)
    {"name": "CD8_T_cell", "type": "cell", "class": "adaptive"},
    {"name": "CD4_T_cell", "type": "cell", "class": "adaptive"},
    {"name": "Macrophage", "type": "cell", "class": "innate"},
    {"name": "Dendritic_cell", "type": "cell", "class": "innate"},
    {"name": "NK_cell", "type": "cell", "class": "innate"},
]

for mol in immune_molecules:
    G.add_vertex(
        f"immune_{mol['name']}",
        entity_type="immune",
        name=mol["name"],
        molecule_type=mol["type"],
        immune_class=mol["class"]
    )

print(f"Added {len(immune_molecules)} immune molecules")

Added 17 immune molecules


In [12]:
# ### 4.4 Metabolites

metabolites = [
    {"name": "glucose", "pathway": "glycolysis", "role": "substrate"},
    {"name": "G6P", "pathway": "glycolysis", "role": "intermediate"},
    {"name": "F6P", "pathway": "glycolysis", "role": "intermediate"},
    {"name": "FBP", "pathway": "glycolysis", "role": "intermediate"},
    {"name": "pyruvate", "pathway": "glycolysis", "role": "product"},
    {"name": "lactate", "pathway": "glycolysis", "role": "product"},
    {"name": "acetyl_CoA", "pathway": "TCA", "role": "intermediate"},
    {"name": "citrate", "pathway": "TCA", "role": "intermediate"},
    {"name": "ATP", "pathway": "energy", "role": "currency"},
    {"name": "ADP", "pathway": "energy", "role": "currency"},
    {"name": "NAD", "pathway": "redox", "role": "cofactor"},
    {"name": "NADH", "pathway": "redox", "role": "cofactor"},
    {"name": "malonyl_CoA", "pathway": "lipid", "role": "intermediate"},
    {"name": "palmitate", "pathway": "lipid", "role": "product"},
    {"name": "cholesterol", "pathway": "lipid", "role": "product"},
    {"name": "phospholipid", "pathway": "lipid", "role": "product"},
]

for met in metabolites:
    G.add_vertex(
        f"met_{met['name']}",
        entity_type="metabolite",
        name=met["name"],
        pathway=met["pathway"],
        metabolic_role=met["role"]
    )

print(f"Added {len(metabolites)} metabolites")

Added 16 metabolites


In [13]:
# ### 4.5 Phenotypes & Outcomes

phenotypes = [
    {"name": "viral_replication_rate", "category": "viral"},
    {"name": "viral_titer", "category": "viral"},
    {"name": "cell_death", "category": "pathology"},
    {"name": "inflammation", "category": "pathology"},
    {"name": "cytokine_storm", "category": "pathology"},
    {"name": "T_cell_exhaustion", "category": "immune"},
    {"name": "antibody_response", "category": "immune"},
    {"name": "tissue_damage", "category": "pathology"},
    {"name": "recovery", "category": "outcome"},
    {"name": "severe_disease", "category": "outcome"},
]

for pheno in phenotypes:
    G.add_vertex(
        f"pheno_{pheno['name']}",
        entity_type="phenotype",
        name=pheno["name"],
        category=pheno["category"]
    )

print(f"Added {len(phenotypes)} phenotypes")
G.mark("entities_added")

Added 10 phenotypes


In [14]:
# ## 5. Define Layer Presence (V_M)

# host signaling layer
for v in G.vertices():
    attrs = G.get_vertex_attrs(v)
    etype = attrs.get("entity_type")
    
    if etype == "host_protein":
        func = attrs.get("function", "")
        if func in ["transcription_factor", "kinase", "adaptor", "sensor", "protease", "viral_receptor"]:
            G.add_presence(v, ("host_signaling",))
    
    if etype == "viral_protein":
        # viral proteins that interact with host signaling
        if attrs.get("function") in ["immune_evasion", "host_shutoff"]:
            G.add_presence(v, ("host_signaling",))

# viral replication layer
for v in G.vertices():
    attrs = G.get_vertex_attrs(v)
    etype = attrs.get("entity_type")
    
    if etype == "viral_protein":
        G.add_presence(v, ("viral_replication",))
    
    if etype == "host_protein":
        # host factors needed for replication
        func = attrs.get("function", "")
        if func in ["translation", "ribosome", "protease"]:
            G.add_presence(v, ("viral_replication",))

# immune response layer
for v in G.vertices():
    attrs = G.get_vertex_attrs(v)
    etype = attrs.get("entity_type")
    
    if etype == "immune":
        G.add_presence(v, ("immune_response",))
    
    if etype == "host_protein":
        func = attrs.get("function", "")
        if func in ["transcription_factor", "sensor", "adaptor"]:
            G.add_presence(v, ("immune_response",))
    
    if etype == "viral_protein":
        if attrs.get("function") == "immune_evasion":
            G.add_presence(v, ("immune_response",))

# metabolic layer  
for v in G.vertices():
    attrs = G.get_vertex_attrs(v)
    etype = attrs.get("entity_type")
    
    if etype == "metabolite":
        G.add_presence(v, ("metabolic",))
    
    if etype == "host_protein":
        func = attrs.get("function", "")
        if func in ["glycolysis", "lipid_synthesis"]:
            G.add_presence(v, ("metabolic",))
    
    if etype == "viral_protein":
        # viral proteins that hijack metabolism
        if attrs.get("name") in ["NSP3", "NSP12"]:
            G.add_presence(v, ("metabolic",))

# transcriptional layer
for v in G.vertices():
    attrs = G.get_vertex_attrs(v)
    etype = attrs.get("entity_type")
    
    if etype == "host_protein":
        func = attrs.get("function", "")
        if func == "transcription_factor":
            G.add_presence(v, ("transcriptional",))
    
    if etype == "viral_protein":
        if attrs.get("function") in ["host_shutoff", "methyltransferase"]:
            G.add_presence(v, ("transcriptional",))
    
    if etype == "phenotype":
        G.add_presence(v, ("transcriptional",))

# phenotypes also in immune layer
for v in G.vertices():
    attrs = G.get_vertex_attrs(v)
    if attrs.get("entity_type") == "phenotype":
        G.add_presence(v, ("immune_response",))

print("Layer presence summary:")
for layer in G.elem_layers["biological_context"]:
    verts = G.layer_vertex_set((layer,))
    print(f"  {layer}: {len(verts)} entities")

Layer presence summary:
  host_signaling: 17 entities
  viral_replication: 22 entities
  immune_response: 38 entities
  metabolic: 25 entities
  transcriptional: 17 entities


In [15]:
# ## 6. Add Edge Entities (Reactions as Nodes)
# 
# Metabolic reactions are modeled as edge-entities that connect 
# substrates → reaction → products.

# define metabolic reactions as edge entities
reactions = [
    {
        "id": "rxn_HK2",
        "enzyme": "host_HK2",
        "substrates": ["met_glucose", "met_ATP"],
        "products": ["met_G6P", "met_ADP"],
        "reversible": False,
        "stoich": {"met_glucose": -1, "met_ATP": -1, "met_G6P": 1, "met_ADP": 1}
    },
    {
        "id": "rxn_PFK",
        "enzyme": "host_PFKFB3",
        "substrates": ["met_F6P", "met_ATP"],
        "products": ["met_FBP", "met_ADP"],
        "reversible": False,
        "stoich": {"met_F6P": -1, "met_ATP": -1, "met_FBP": 1, "met_ADP": 1}
    },
    {
        "id": "rxn_PK",
        "enzyme": "host_PKM2", 
        "substrates": ["met_FBP", "met_ADP"],
        "products": ["met_pyruvate", "met_ATP"],
        "reversible": False,
        "stoich": {"met_FBP": -1, "met_ADP": -1, "met_pyruvate": 1, "met_ATP": 1}
    },
    {
        "id": "rxn_LDH",
        "enzyme": "host_LDHA",
        "substrates": ["met_pyruvate", "met_NADH"],
        "products": ["met_lactate", "met_NAD"],
        "reversible": True,
        "stoich": {"met_pyruvate": -1, "met_NADH": -1, "met_lactate": 1, "met_NAD": 1}
    },
    {
        "id": "rxn_ACC",
        "enzyme": "host_ACC1",
        "substrates": ["met_acetyl_CoA", "met_ATP"],
        "products": ["met_malonyl_CoA", "met_ADP"],
        "reversible": False,
        "stoich": {"met_acetyl_CoA": -1, "met_ATP": -1, "met_malonyl_CoA": 1, "met_ADP": 1}
    },
    {
        "id": "rxn_FAS",
        "enzyme": "host_FASN",
        "substrates": ["met_malonyl_CoA", "met_NADH"],
        "products": ["met_palmitate", "met_NAD"],
        "reversible": False,
        "stoich": {"met_malonyl_CoA": -7, "met_NADH": -14, "met_palmitate": 1, "met_NAD": 14}
    },
]

# add reactions as edge entities
for rxn in reactions:
    G.add_edge_entity(
        rxn["id"],
        entity_type="reaction",
        name=rxn["id"],
        enzyme=rxn["enzyme"],
        reversible=rxn["reversible"]
    )
    # add to metabolic layer
    G._slices["default"]["edges"].add(rxn["id"])

print(f"Added {len(reactions)} reaction edge-entities")

Added 6 reaction edge-entities


In [16]:
# ## 7. Build Intra-Layer Edges

# === HOST SIGNALING LAYER ===

host_signaling_edges = [
    # viral entry
    ("host_ACE2", "viral_Spike", 0.95, True),  # receptor binding
    ("host_TMPRSS2", "viral_Spike", 0.85, True),  # spike cleavage
    ("host_FURIN", "viral_Spike", 0.80, True),
    
    # pattern recognition
    ("host_RIG_I", "host_MAVS", 0.9, True),
    ("host_MDA5", "host_MAVS", 0.9, True),
    ("host_MAVS", "host_TBK1", 0.85, True),
    ("host_MAVS", "host_IKKe", 0.85, True),
    
    # kinase cascade
    ("host_TBK1", "host_IRF3", 0.9, True),
    ("host_TBK1", "host_IRF7", 0.8, True),
    ("host_IKKe", "host_NFkB", 0.85, True),
    
    # viral interference
    ("viral_NSP1", "host_STAT1", 0.7, True),  # inhibition
    ("viral_NSP1", "host_STAT2", 0.7, True),
    ("viral_ORF6", "host_STAT1", 0.8, True),
    ("viral_ORF6", "host_IRF3", 0.75, True),
]

for src, tgt, w, directed in host_signaling_edges:
    if G.has_presence(src, ("host_signaling",)) and G.has_presence(tgt, ("host_signaling",)):
        G.add_intra_edge_nl(src, tgt, ("host_signaling",), weight=w)

# === VIRAL REPLICATION LAYER ===

viral_replication_edges = [
    # replication complex
    ("viral_NSP12", "viral_NSP13", 0.95, False),  # RdRp-helicase
    ("viral_NSP12", "viral_NSP14", 0.9, False),
    ("viral_NSP14", "viral_NSP16", 0.85, False),
    ("viral_NSP3", "viral_NSP5", 0.9, False),  # proteases
    
    # structural assembly
    ("viral_Nucleocapsid", "viral_Membrane", 0.8, False),
    ("viral_Membrane", "viral_Envelope", 0.85, False),
    ("viral_Spike", "viral_Membrane", 0.9, False),
    
    # host factor hijacking
    ("viral_NSP1", "host_eIF4E", 0.8, True),  # translation shutoff
    ("viral_NSP1", "host_eIF4G", 0.75, True),
    ("viral_Nucleocapsid", "host_PABP", 0.7, False),
]

for src, tgt, w, directed in viral_replication_edges:
    if G.has_presence(src, ("viral_replication",)) and G.has_presence(tgt, ("viral_replication",)):
        G.add_intra_edge_nl(src, tgt, ("viral_replication",), weight=w)

# === IMMUNE RESPONSE LAYER ===

immune_edges = [
    # IFN signaling
    ("host_IRF3", "immune_IFN_beta", 0.9, True),
    ("host_IRF7", "immune_IFN_alpha", 0.9, True),
    ("immune_IFN_alpha", "immune_IFNAR1", 0.85, True),
    ("immune_IFN_beta", "immune_IFNAR1", 0.85, True),
    ("immune_IFNAR1", "immune_IFNAR2", 0.9, False),  # receptor complex
    ("immune_IFNAR1", "host_STAT1", 0.8, True),
    ("immune_IFNAR2", "host_STAT2", 0.8, True),
    
    # proinflammatory
    ("host_NFkB", "immune_IL6", 0.85, True),
    ("host_NFkB", "immune_IL1beta", 0.8, True),
    ("host_NFkB", "immune_TNF_alpha", 0.85, True),
    
    # immune cell activation
    ("immune_IFN_gamma", "immune_Macrophage", 0.8, True),
    ("immune_IL6", "immune_CD4_T_cell", 0.7, True),
    ("immune_CXCL10", "immune_CD8_T_cell", 0.75, True),
    ("immune_IFN_alpha", "immune_NK_cell", 0.8, True),
    ("immune_IFN_alpha", "immune_Dendritic_cell", 0.85, True),
    
    # TLR sensing
    ("immune_TLR3", "host_IRF3", 0.8, True),
    ("immune_TLR7", "host_IRF7", 0.8, True),
    
    # viral evasion
    ("viral_ORF6", "immune_IFN_beta", 0.6, True),  # blocks IFN
    ("viral_ORF7a", "immune_CD8_T_cell", 0.5, True),  # impairs T cells
    ("viral_ORF8", "immune_Macrophage", 0.55, True),
    
    # phenotype links
    ("immune_IL6", "pheno_inflammation", 0.9, True),
    ("immune_TNF_alpha", "pheno_inflammation", 0.85, True),
    ("immune_IL6", "pheno_cytokine_storm", 0.8, True),
    ("immune_IL1beta", "pheno_cytokine_storm", 0.75, True),
    ("immune_CD8_T_cell", "pheno_antibody_response", 0.7, True),
    ("pheno_cytokine_storm", "pheno_severe_disease", 0.9, True),
    ("pheno_inflammation", "pheno_tissue_damage", 0.8, True),
]

for src, tgt, w, directed in immune_edges:
    if G.has_presence(src, ("immune_response",)) and G.has_presence(tgt, ("immune_response",)):
        G.add_intra_edge_nl(src, tgt, ("immune_response",), weight=w)

# === TRANSCRIPTIONAL LAYER ===

transcriptional_edges = [
    # TF -> target gene expression (simplified)
    ("host_STAT1", "host_IRF7", 0.85, True),
    ("host_STAT2", "host_RIG_I", 0.8, True),
    ("host_IRF3", "host_STAT1", 0.75, True),
    ("host_NFkB", "host_STAT1", 0.7, True),
    
    # viral shutoff of transcription
    ("viral_NSP1", "host_STAT1", 0.8, True),
    ("viral_NSP16", "host_IRF3", 0.6, True),  # cap methylation evasion
    
    # phenotype regulation
    ("host_NFkB", "pheno_inflammation", 0.85, True),
    ("host_IRF3", "pheno_antibody_response", 0.7, True),
    ("host_STAT1", "pheno_recovery", 0.75, True),
]

for src, tgt, w, directed in transcriptional_edges:
    if G.has_presence(src, ("transcriptional",)) and G.has_presence(tgt, ("transcriptional",)):
        G.add_intra_edge_nl(src, tgt, ("transcriptional",), weight=w)

print(f"Total edges after intra-layer: {G.number_of_edges()}")
G.mark("intra_layer_edges_added")

Total edges after intra-layer: 56


In [17]:
# ## 8. Add Hyperedges (Protein Complexes & Multi-Component Reactions)

# === PROTEIN COMPLEXES (undirected hyperedges) ===

complexes = [
    {
        "name": "RTC",  # replication-transcription complex
        "members": ["viral_NSP12", "viral_NSP13", "viral_NSP14", "viral_NSP16"],
        "function": "viral_replication",
        "essential": True
    },
    {
        "name": "IFNAR_complex",
        "members": ["immune_IFNAR1", "immune_IFNAR2", "host_STAT1", "host_STAT2"],
        "function": "IFN_signaling",
        "essential": True
    },
    {
        "name": "inflammasome",
        "members": ["host_NFkB", "immune_IL1beta", "immune_IL6"],
        "function": "inflammation",
        "essential": False
    },
    {
        "name": "translation_initiation",
        "members": ["host_eIF4E", "host_eIF4G", "host_PABP"],
        "function": "translation",
        "essential": True
    },
    {
        "name": "viral_entry_complex",
        "members": ["viral_Spike", "host_ACE2", "host_TMPRSS2"],
        "function": "viral_entry",
        "essential": True
    },
    {
        "name": "virion_assembly",
        "members": ["viral_Spike", "viral_Membrane", "viral_Envelope", "viral_Nucleocapsid"],
        "function": "virion_production",
        "essential": True
    },
]

for cplx in complexes:
    # verify all members exist
    valid_members = [m for m in cplx["members"] if m in G.entity_to_idx]
    if len(valid_members) >= 2:
        G.add_hyperedge(
            members=valid_members,
            edge_id=f"complex_{cplx['name']}",
            weight=1.0,
            complex_name=cplx["name"],
            function=cplx["function"],
            essential=cplx["essential"]
        )

print(f"Added {len(complexes)} protein complexes as hyperedges")

# === DIRECTED HYPEREDGES (multi-substrate reactions) ===

directed_reactions = [
    {
        "name": "glycolysis_committed_step",
        "head": ["met_F6P", "met_ATP", "host_PFKFB3"],  # substrates + enzyme
        "tail": ["met_FBP", "met_ADP"],  # products
        "pathway": "glycolysis"
    },
    {
        "name": "fatty_acid_synthesis",
        "head": ["met_malonyl_CoA", "met_NADH", "host_FASN"],
        "tail": ["met_palmitate", "met_NAD"],
        "pathway": "lipid"
    },
    {
        "name": "IFN_production",
        "head": ["host_IRF3", "host_IRF7", "host_NFkB"],
        "tail": ["immune_IFN_alpha", "immune_IFN_beta"],
        "pathway": "immune"
    },
    {
        "name": "viral_genome_replication",
        "head": ["viral_NSP12", "viral_NSP13", "met_ATP"],
        "tail": ["pheno_viral_replication_rate"],
        "pathway": "viral"
    },
    {
        "name": "cytokine_storm_induction",
        "head": ["immune_IL6", "immune_IL1beta", "immune_TNF_alpha"],
        "tail": ["pheno_cytokine_storm", "pheno_severe_disease"],
        "pathway": "pathology"
    },
]

for rxn in directed_reactions:
    valid_head = [h for h in rxn["head"] if h in G.entity_to_idx]
    valid_tail = [t for t in rxn["tail"] if t in G.entity_to_idx]
    
    if len(valid_head) >= 1 and len(valid_tail) >= 1:
        G.add_hyperedge(
            head=valid_head,
            tail=valid_tail,
            edge_id=f"hrxn_{rxn['name']}",
            weight=1.0,
            reaction_name=rxn["name"],
            pathway=rxn["pathway"]
        )

print(f"Added {len(directed_reactions)} directed hyperedges")
print(f"Total hyperedges: {len(G.hyperedge_definitions)}")

Added 6 protein complexes as hyperedges
Added 5 directed hyperedges
Total hyperedges: 11


In [18]:
# ## 9. Add Edge-Entity Connections (Reaction Networks)
# 
# Connect substrates → reaction → products using vertex-edge edges.

for rxn in reactions:
    rxn_id = rxn["id"]
    enzyme = rxn["enzyme"]
    
    # enzyme catalyzes reaction
    if enzyme in G.entity_to_idx:
        G.add_edge(enzyme, rxn_id, edge_type="vertex_edge", 
                   weight=1.0, relation="catalyzes")
    
    # substrates feed into reaction
    for sub in rxn["substrates"]:
        if sub in G.entity_to_idx:
            G.add_edge(sub, rxn_id, edge_type="vertex_edge",
                       weight=abs(rxn["stoich"].get(sub, 1)), 
                       relation="substrate")
    
    # reaction produces products
    for prod in rxn["products"]:
        if prod in G.entity_to_idx:
            G.add_edge(rxn_id, prod, edge_type="vertex_edge",
                       weight=abs(rxn["stoich"].get(prod, 1)),
                       relation="product")

# set stoichiometric coefficients on hyperedges
for rxn in reactions:
    rxn_id = rxn["id"]
    if rxn_id in G.edge_to_idx:
        # coefficients already encoded in edge weights above
        pass

print(f"Total edges after edge-entity connections: {G.number_of_edges()}")
G.mark("edge_entities_connected")

Total edges after edge-entity connections: 97


In [20]:
# ## 10. Add Inter-Layer Coupling Edges

omega = 1.0

valid_layers = set(G.elem_layers["biological_context"])

coupling_count = 0

for v in G.vertices():
    # get all layers the vertex is present in
    layers_present = [L for L in G.iter_vertex_layers(v) if L[0] in valid_layers]

    # need ≥2 layers to couple
    for i in range(len(layers_present)):
        for j in range(i+1, len(layers_present)):
            L1 = layers_present[i]
            L2 = layers_present[j]
            G.add_coupling_edge_nl(v, L1, L2, weight=omega)
            coupling_count += 1

print(f"Added {coupling_count} coupling edges")

print(f"Total edges: {G.number_of_edges()}")
G.mark("coupling_edges_added")

Added 45 coupling edges
Total edges: 142


In [21]:
# ## 11. Add Per-Slice Context-Dependent Weights
# 
# Create infection-stage slices with different edge weights.

# create temporal/contextual slices
infection_stages = [
    {"id": "early_infection", "hours": "0-6", "description": "viral entry and initial replication"},
    {"id": "peak_replication", "hours": "6-24", "description": "maximum viral load"},
    {"id": "immune_activation", "hours": "24-72", "description": "innate immune response peak"},
    {"id": "adaptive_response", "hours": "72-168", "description": "T cell and antibody response"},
    {"id": "resolution", "hours": "168+", "description": "viral clearance or chronic infection"},
]

for stage in infection_stages:
    G.add_slice(stage["id"], 
                hours=stage["hours"], 
                description=stage["description"])

# add stage-specific edge weights
# early infection: viral entry high, immune low
early_weights = {
    "viral_Spike": {"host_ACE2": 1.0, "host_TMPRSS2": 1.0},
    "immune_IFN_alpha": {"immune_IFNAR1": 0.2},  # IFN not yet induced
}

# peak replication: replication high, immune starting
peak_weights = {
    "viral_NSP12": {"viral_NSP13": 1.0},
    "viral_NSP1": {"host_eIF4E": 0.9},  # strong host shutoff
    "immune_IFN_beta": {"immune_IFNAR1": 0.5},
}

# immune activation: cytokines high
immune_weights = {
    "host_NFkB": {"immune_IL6": 1.0, "immune_TNF_alpha": 1.0},
    "immune_IL6": {"pheno_inflammation": 1.0},
    "viral_ORF6": {"immune_IFN_beta": 0.8},  # viral evasion active
}

# apply weights to slices (simplified - would normally iterate all edges)
print("Created infection stage slices with context-dependent weights")

Created infection stage slices with context-dependent weights


In [22]:
# ## 12. Build Supra-Adjacency and Analyze

# build vertex-layer index
n_supra = G.ensure_vertex_layer_index()
print(f"Supra-graph nodes: {n_supra}")

# supra-adjacency
A_supra = G.supra_adjacency()
print(f"Supra-adjacency: shape={A_supra.shape}, nnz={A_supra.nnz}")

# supra-laplacian
L_supra = G.supra_laplacian(kind="comb")
print(f"Supra-Laplacian computed")

# eigenanalysis
k = min(12, n_supra - 1)
eigenvalues, eigenvectors = eigsh(L_supra.astype(float), k=k, which="SM")
idx = np.argsort(eigenvalues)
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print(f"\nSmallest eigenvalues:")
for i in range(min(6, len(eigenvalues))):
    print(f"  λ_{i} = {eigenvalues[i]:.6f}")

lambda2 = eigenvalues[1]
print(f"\nAlgebraic connectivity: {lambda2:.6f}")

Supra-graph nodes: 202
Supra-adjacency: shape=(202, 202), nnz=202
Supra-Laplacian computed

Smallest eigenvalues:
  λ_0 = -0.000000
  λ_1 = -0.000000
  λ_2 = -0.000000
  λ_3 = -0.000000
  λ_4 = 0.033727
  λ_5 = 0.044039

Algebraic connectivity: -0.000000


In [29]:
# ## 13. NetworkX Algorithm Integration

# NetworkX for algorithms

# degree centrality
degree_cent = G.nx.degree_centrality(G)
top_degree = sorted(degree_cent.items(), key=lambda x: -x[1])[:10]
print("\nTop 10 nodes by degree centrality:")
for node, cent in top_degree:
    print(f"  {node}: {cent:.4f}")

# betweenness centrality
betweenness = G.nx.betweenness_centrality(G, k=min(100, G.number_of_vertices()))
top_between = sorted(betweenness.items(), key=lambda x: -x[1])[:10]
print("\nTop 10 nodes by betweenness centrality:")
for node, cent in top_between:
    print(f"  {node}: {cent:.4f}")

# connected components
components = list(G.nx.weakly_connected_components(G))
print(f"\nConnected components: {len(components)}")
print(f"Largest component size: {len(max(components, key=len))}")

# pagerank
pagerank = G.nx.pagerank(G, alpha=0.85)
top_pr = sorted(pagerank.items(), key=lambda x: -x[1])[:10]
print("\nTop 10 nodes by PageRank:")
for node, pr in top_pr:
    print(f"  {node}: {pr:.4f}")


Top 10 nodes by degree centrality:
  3: 0.2273
  7: 0.2045
  5: 0.1705
  4: 0.1591
  6: 0.1364
  24: 0.1250
  28: 0.1250
  31: 0.1250
  35: 0.1136
  43: 0.1136

Top 10 nodes by betweenness centrality:
  85: 0.0186
  65: 0.0123
  3: 0.0120
  66: 0.0109
  7: 0.0107
  5: 0.0098
  87: 0.0086
  40: 0.0072
  8: 0.0064
  83: 0.0057

Connected components: 15
Largest component size: 72

Top 10 nodes by PageRank:
  82: 0.0606
  79: 0.0596
  73: 0.0469
  81: 0.0406
  80: 0.0392
  3: 0.0344
  4: 0.0332
  48: 0.0254
  85: 0.0244
  22: 0.0237


In [47]:
# ## 14. Community Detection

from annnet.adapters.networkx_adapter import to_nx
nxG, man = to_nx(G, directed=True, hyperedge_mode="expand")

# convert to undirected for community detection
nx_undir = nxG.to_undirected()

# Louvain communities
try:
    communities = nx.community.louvain_communities(nx_undir, seed=42)
    print(f"Louvain detected {len(communities)} communities")
    
    # analyze community composition
    for i, comm in enumerate(communities[:5]):  # show first 5
        type_counts = defaultdict(int)
    
        for node in comm:
            attrs = nx_undir.nodes[node]     # <--- correct
            etype = attrs.get("entity_type", "unknown")
            type_counts[etype] += 1
    
        print(f"\nCommunity {i} ({len(comm)} nodes):")
        for etype, count in sorted(type_counts.items(), key=lambda x: -x[1]):
            print(f"  {etype}: {count}")
except Exception as e:
    print(f"Community detection failed: {e}")

Louvain detected 23 communities

Community 0 (1 nodes):
  host_protein: 1

Community 1 (9 nodes):
  immune: 6
  host_protein: 2
  viral_protein: 1

Community 2 (5 nodes):
  host_protein: 5

Community 3 (10 nodes):
  metabolite: 6
  host_protein: 2
  unknown: 2

Community 4 (1 nodes):
  host_protein: 1


In [48]:
# ## 15. Shortest Paths Analysis

# find shortest paths from viral entry to disease outcomes
source = "viral_Spike"
targets = ["pheno_severe_disease", "pheno_cytokine_storm", "pheno_recovery"]

print("Shortest paths from viral_Spike to disease outcomes:")
for target in targets:
    if target in nxG and source in nxG:
        try:
            if nx.has_path(nxG, source, target):
                path = nx.shortest_path(nxG, source, target)
                print(f"\n  → {target} (length {len(path)-1}):")
                print(f"    {' → '.join(path)}")
            else:
                print(f"\n  → {target}: No path exists")
        except nx.NetworkXError as e:
            print(f"\n  → {target}: {e}")

Shortest paths from viral_Spike to disease outcomes:

  → pheno_severe_disease: No path exists

  → pheno_cytokine_storm: No path exists

  → pheno_recovery: No path exists


In [49]:
# ## 16. Diffusion Simulation: Drug Target Perturbation

def simulate_perturbation(G, target_node, target_layer, n_steps=50, tau=0.1):
    """Simulate perturbation (e.g., drug inhibition) of a target."""
    G.ensure_vertex_layer_index()
    n = len(G._row_to_nl)
    L = G.supra_laplacian(kind="comb")
    
    # initial state: all zeros except -1 at target (inhibition)
    x = np.zeros(n)
    for i, (v, layer) in enumerate(G._row_to_nl):
        if v == target_node and layer == (target_layer,):
            x[i] = -1.0  # inhibition signal
            break
    
    # diffuse
    trajectory = [x.copy()]
    for _ in range(n_steps):
        x = x - tau * (L @ x)
        trajectory.append(x.copy())
    
    return np.array(trajectory)

# simulate NSP5 (main protease) inhibition - like Paxlovid
target = "viral_NSP5"
target_layer = "viral_replication"

if G.has_presence(target, (target_layer,)):
    traj = simulate_perturbation(G, target, target_layer, n_steps=50)
    
    # analyze effect on phenotypes
    print(f"Drug target perturbation: {target} in {target_layer}")
    print("\nEffect on phenotypes after diffusion:")
    
    for i, (v, layer) in enumerate(G._row_to_nl):
        if "pheno_" in v:
            initial = traj[0, i]
            final = traj[-1, i]
            if abs(final) > 0.001:
                direction = "↓" if final < 0 else "↑"
                print(f"  {v} ({layer[0]}): {final:.4f} {direction}")

Drug target perturbation: viral_NSP5 in viral_replication

Effect on phenotypes after diffusion:


In [51]:
# ## 17. Export to Cytoscape (CX2 Format)
from annnet.adapters import cx2_adapter as cx
import json

virus = cx.to_cx2(G)


output_file = "Virus.cx2"

with open(output_file, "w") as f:
    json.dump(virus, f)

print(f"Saved to {output_file}")

Saved to Virus.cx2


In [54]:
# ## 18. Export to Native .annnet Format

# the native format preserves everything: hyperedges, layers, history, etc.
G.write("viral_infection_model.annnet", overwrite=True )
print("Saved to viral_infection_model.annnet (lossless)")

Saved to viral_infection_model.annnet (lossless)


In [55]:
# ## 19. History & Audit Trail

# view mutation history
history = G.history(as_df=True)
print(f"Total mutations logged: {len(history)}")
print(f"\nHistory columns: {history.columns}")

# show marked checkpoints
marks = history.filter(pl.col("op") == "mark")
print(f"\nCheckpoints:")
print(marks.select(["version", "ts_utc", "label"]))

# operation counts
op_counts = history.group_by("op").agg(pl.count().alias("count")).sort("count", descending=True)
print(f"\nOperations by type:")
print(op_counts.head(10))

# export history
try:
    n_exported = G.export_history("viral_model_history.parquet")
    print(f"\nExported {n_exported} history events to viral_model_history.parquet")
except Exception as e:
    print(f"History export: {e}")

Total mutations logged: 344

History columns: ['version', 'ts_utc', 'mono_ns', 'op', 'label', 'vertex_id', 'slice', 'attributes', 'result', 'edge_id', 'attrs', 'edge_entity_id', 'source', 'target', 'weight', 'edge_type', 'propagate', 'slice_weight', 'directed', 'edge_directed']

Checkpoints:
shape: (5, 3)
┌─────────┬─────────────────────────────┬─────────────────────────┐
│ version ┆ ts_utc                      ┆ label                   │
│ ---     ┆ ---                         ┆ ---                     │
│ i64     ┆ str                         ┆ str                     │
╞═════════╪═════════════════════════════╪═════════════════════════╡
│ 1       ┆ 2025-12-04T12:57:03.998688Z ┆ initialization_start    │
│ 85      ┆ 2025-12-04T12:57:10.466513Z ┆ entities_added          │
│ 210     ┆ 2025-12-04T12:57:15.937495Z ┆ intra_layer_edges_added │
│ 293     ┆ 2025-12-04T12:57:18.035288Z ┆ edge_entities_connected │
│ 339     ┆ 2025-12-04T13:00:01.660277Z ┆ coupling_edges_added    │
└─────────┴──

/tmp/ipykernel_3673/2238291244.py:14: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  op_counts = history.group_by("op").agg(pl.count().alias("count")).sort("count", descending=True)


In [56]:
# ## 20. Snapshots & Diffing

# create snapshot of current state
snap_final = G.snapshot(label="final_model")
print(f"Snapshot created: {snap_final['label']}")
print(f"  Vertices: {snap_final['counts']['vertices']}")
print(f"  Edges: {snap_final['counts']['edges']}")
print(f"  Slices: {snap_final['counts']['slices']}")

# list all snapshots
print(f"\nAll snapshots:")
for snap in G.list_snapshots():
    print(f"  {snap['label']} (v{snap['version']}): {snap['counts']}")

Snapshot created: final_model
  Vertices: 83
  Edges: 142
  Slices: 6

All snapshots:
  final_model (v344): {'vertices': 83, 'edges': 142, 'slices': 6}


In [57]:
# ## 21. Composite Key Lookup Demo

# lookup entities by attributes instead of ID
print("Composite key lookups:")

# find viral proteins by name
try:
    spike = G.get_vertex_by_attrs(entity_type="viral_protein", name="Spike")
    print(f"  Spike protein vertex ID: {spike}")
except Exception as e:
    print(f"  Lookup not available: {e}")

# find all immune cytokines
cytokines = [v for v in G.vertices() 
             if G.get_vertex_attrs(v).get("molecule_type") == "cytokine"]
print(f"  Cytokines found: {len(cytokines)}")
for c in cytokines[:5]:
    print(f"    - {c}")



Composite key lookups:
  Lookup not available: 'Graph' object has no attribute 'get_vertex_by_attrs'
  Cytokines found: 7
    - immune_IFN_alpha
    - immune_IFN_beta
    - immune_IFN_gamma
    - immune_IL6
    - immune_IL1beta


In [60]:
# ## 23. Final Summary & Statistics

print("=" * 60)
print("VIRAL INFECTION MULTILAYER MODEL - SUMMARY")
print("=" * 60)

# entity counts
entity_counts = defaultdict(int)
for v in G.vertices():
    etype = G.get_vertex_attrs(v).get("entity_type", "unknown")
    entity_counts[etype] += 1

print("\nEntity counts:")
for etype, count in sorted(entity_counts.items()):
    print(f"  {etype}: {count}")

# edge statistics
print(f"\nEdge statistics:")
print(f"  Total edges: {G.number_of_edges()}")
print(f"  Binary edges: {len(G.edge_definitions)}")
print(f"  Hyperedges: {len(G.hyperedge_definitions)}")
print(f"  Edge entities: {len([e for e, t in G.entity_types.items() if t == 'edge'])}")

# layer statistics
print(f"\nLayer statistics:")
for layer in G.elem_layers["biological_context"]:
    n_verts = len(G.layer_vertex_set((layer,)))
    n_edges = len(G.layer_edge_set((layer,)))
    print(f"  {layer}: {n_verts} vertices, {n_edges} intra-layer edges")

# coupling statistics
coupling_edges = [e for e, k in G.edge_kind.items() if k == "coupling"]
print(f"\nCoupling edges: {len(coupling_edges)}")

# supra statistics
print(f"\nSupra-graph:")
print(f"  Nodes (vertex-layer pairs): {n_supra}")
print(f"  Algebraic connectivity: {lambda2:.6f}")

# slice statistics
print(f"\nSlices: {len(G.list_slices())}")
for sid in G.list_slices():
    info = G.get_slice_info(sid)
    print(f"  {sid}: {len(info['vertices'])} V, {len(info['edges'])} E")

# memory usage
mem = G.memory_usage()
print(f"\nEstimated memory usage: {mem / 1024:.2f} KB")

# history
print(f"\nMutation history: {len(G._history)} events")
print(f"Current version: {G._version}")

print("\n" + "=" * 60)
print("Model complete. Files exported:")
print("  - viral_infection_model.cx2 (Cytoscape)")
print("  - viral_infection_model.annnet (native)")
print("  - viral_model_history.parquet (audit trail)")
print("=" * 60)

VIRAL INFECTION MULTILAYER MODEL - SUMMARY

Entity counts:
  host_protein: 24
  immune: 17
  metabolite: 16
  phenotype: 10
  viral_protein: 16

Edge statistics:
  Total edges: 142
  Binary edges: 142
  Hyperedges: 11
  Edge entities: 6

Layer statistics:
  host_signaling: 17 vertices, 11 intra-layer edges
  viral_replication: 22 vertices, 10 intra-layer edges
  immune_response: 38 vertices, 27 intra-layer edges
  metabolic: 25 vertices, 0 intra-layer edges
  transcriptional: 17 vertices, 8 intra-layer edges
  early_infection: 0 vertices, 0 intra-layer edges
  peak_replication: 0 vertices, 0 intra-layer edges
  immune_activation: 0 vertices, 0 intra-layer edges
  adaptive_response: 0 vertices, 0 intra-layer edges
  resolution: 0 vertices, 0 intra-layer edges

Coupling edges: 45

Supra-graph:
  Nodes (vertex-layer pairs): 202
  Algebraic connectivity: -0.000000

Slices: 5
  early_infection: 0 V, 0 E
  peak_replication: 0 V, 0 E
  immune_activation: 0 V, 0 E
  adaptive_response: 0 V, 0 E